# 02 - Data from the Web

We start importing all the libraries that we are going to use.

In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

Generating the base of our futures requests

In [97]:
# We will create a string that contains the url to the isacademia form
# From there we will get all the values we need for making our request petitions.
payloadform = {'ww_i_reportModel': '133685247'}
isa_baseurl = 'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS'
isa_formurl= isa_baseurl + ".filter"
r = requests.get(isa_formurl, params=payloadform)
# Here is the url to the isa form
s = r.url + " " + str(r.status_code)  
s

'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247 200'

In [131]:
soup = BeautifulSoup(r.text, 'html.parser')
form = form = soup.find('form')

def findInputNameAndOptionValue(inputLabel, optionLabel):
    th = form.find('th', string=inputLabel)
    td = th.nextSibling
    input_ = td.find('input')
    option = input_.find('option', string=optionLabel)
    return (input_['name'], option['value'])

findInputNameAndOptionValue("Unité académique", "Informatique")
    

('zz_x_UNITE_ACAD', '249847')

In [132]:
def extractHiddenParameters():
    inputs = form.findAll('input', type='hidden')
    return {input_['name']: input_['value'] for input_ in inputs}

def choose_html():
    input_ = form.find('input', string="html")
    return {input_['name']: input_['value']}

paramsTemplate = {**extractHiddenParameters(), **choose_html()}
paramsTemplate
    

{'ww_b_list': '1',
 'ww_c_langue': '',
 'ww_i_reportModelXsl': '133685270',
 'ww_i_reportmodel': '133685247',
 'zz_x_HIVERETE': '',
 'zz_x_PERIODE_ACAD': '',
 'zz_x_PERIODE_PEDAGO': '',
 'zz_x_UNITE_ACAD': ''}

In [157]:
def generateParams(humanReadableParams):
    params = dict([findInputNameAndOptionValue(x, y) for (x, y) in humanReadableParams.items()])
    return {**paramsTemplate, **params}

# Example

exampleParams = {
    "Unité académique": "Informatique",
    "Période académique": "2015-2016",
    "Période pédagogique": "Master semestre 1",
    "Type de semestre": "Semestre d'automne",
}

generateParams(exampleParams)

{'ww_b_list': '1',
 'ww_c_langue': '',
 'ww_i_reportModelXsl': '133685270',
 'ww_i_reportmodel': '133685247',
 'zz_x_HIVERETE': '2936286',
 'zz_x_PERIODE_ACAD': '213638028',
 'zz_x_PERIODE_PEDAGO': '2230106',
 'zz_x_UNITE_ACAD': '249847'}

With the data that we have just obtain, we will make the request petitions with a loops


In [158]:
def getDataFrame(humanReadableParams):
    r = requests.get(isa_dataurl, params=generateParams(exampleParams))
    assert r.status_code == 200
    return extractTable(r.text)
    
getDataFrame(exampleParams)

,Civilité,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange,No Sciper
0,Monsieur,Abbey Alexandre,None,None,None,None,None,Présent,None,None,235688
1,Monsieur,Ahn Seongho,None,None,None,None,None,Présent,None,None,274015
2,Madame,Alemanno Sara,None,None,None,None,None,Présent,None,None,268410
3,Monsieur,Althaus Luca,None,None,None,None,None,Présent,None,None,271464
4,Monsieur,Assi Karim,None,None,None,None,None,Présent,None,None,274518
5,Monsieur,Badoux Luc-Antoine,None,None,None,None,None,Présent,None,None,249613
6,Monsieur,Bagnoud Jérôme,None,None,None,None,None,Présent,None,None,262214
7,Monsieur,Barbaras Yann Quentin,None,None,None,None,None,Présent,None,None,262239
8,Monsieur,Barras Luca,None,None,None,None,None,Présent,None,None,257916
9,Madame,Barsi Clémence Marie Sabine,None,None,None,None,None,Présent,None,None,271508


In [152]:
# Here we will need to parse the html using BeautifulSoup library.
# The data that we want comes in a html table, it won't be really hard to parse it into pandas dataframe.

# For Bachelor semestre, Master semetre, Project Master the structure of the table is always the same.

def extractTable(html):
    """Extract the table from HTML into a pandas Dataframe."""
    BStable = BeautifulSoup(html, 'html.parser').find('table')

    BSRows = table.findAll('tr')
    # BSrows[0] is just the title of the table, so we ignore it.
    headerBSRow = BSRows[1]
    headers = [th.string for th in headerBSRow]
    # We drop the last column, because it's always empty (originally it is used to make the HTML table look better).
    rows = [[tr.string for tr in BSRow.findAll('td')[:-1]] for BSRow in BSRows[2:]]
    return pd.DataFrame(data=rows, columns=headers)

# Example
extractTable(r.text)

,Civilité,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange,No Sciper
0,Monsieur,Abbey Alexandre,None,None,None,None,None,Présent,None,None,235688
1,Monsieur,Ahn Seongho,None,None,None,None,None,Présent,None,None,274015
2,Madame,Alemanno Sara,None,None,None,None,None,Présent,None,None,268410
3,Monsieur,Althaus Luca,None,None,None,None,None,Présent,None,None,271464
4,Monsieur,Assi Karim,None,None,None,None,None,Présent,None,None,274518
5,Monsieur,Badoux Luc-Antoine,None,None,None,None,None,Présent,None,None,249613
6,Monsieur,Bagnoud Jérôme,None,None,None,None,None,Présent,None,None,262214
7,Monsieur,Barbaras Yann Quentin,None,None,None,None,None,Présent,None,None,262239
8,Monsieur,Barras Luca,None,None,None,None,None,Présent,None,None,257916
9,Madame,Barsi Clémence Marie Sabine,None,None,None,None,None,Présent,None,None,271508
